In [1]:
using Pkg, JuMP, Ipopt, DataFrames, CSV, Random, Distributions, Statistics, HypothesisTests

In [2]:
train_prcnt = 0.7
num_iterations = 4

cweights = CSV.read("C:/Users/Luke/OneDrive/Research/sg_data/SYN_scaled_round_sg_course_weights.csv")
pweights = CSV.read("C:/Users/Luke/OneDrive/Research/sg_data/SYN_scaled_round_sg_plyr_weights.csv")

,Column1,plyr,p_idx,weight,sg_drive_mean,sg_appr_mean,sg_short_mean
,Int64,String,Int64,Float64,Float64,Float64,Float64
1,0,A.J. McInerney,0,0.000703235,-1.64618,0.788106,1.82196
2,1,Aaron Baddeley,1,0.00464135,-0.138715,-0.310845,-0.399682
3,2,Aaron Wise,2,0.00548523,-0.259793,-0.138403,0.30461
4,3,Abraham Ancer,3,0.00583685,0.17854,-0.181372,-0.30378
5,4,Adam Bland,4,0.000281294,-1.06166,0.0535213,1.22175
6,5,Adam D'Amario,5,0.000140647,-0.588041,-0.925146,0.831257
7,6,Adam Hadwin,6,0.0045007,-0.000717188,-0.366841,0.177007
8,7,Adam Schenk,7,0.00464135,0.0422463,0.187723,-0.0758222
9,8,Adam Scott,8,0.00407876,0.00926677,0.196328,-0.137729


In [3]:
function get_weights(df, pweight_df = pweights,cweight_df = cweights)
    #println("test1")
    n = nrow(df)
    p_d=Dict([(i,count(x->x==i,df[!,:p_idx])/n) for i in unique(df[!,:p_idx])])
    p_weight = DataFrame(p_idx = collect(keys(p_d)), weight = collect(values(p_d)))
    #println("test2")
    c_d = Dict([(i,count(x->x==i,df[!,:c_idx])/n) for i in unique(df[!,:c_idx])])
    c_weight = DataFrame(c_idx = collect(keys(c_d)), weight = collect(values(c_d)))
    #println("test3")
    p_drive_cnstr = Dict([(i,pweight_df[!,:sg_drive_mean][i+1]) for i in unique(df[!,:p_idx])])
    #println("test4")
    p_appr_cnstr = Dict([(i,pweight_df[!,:sg_appr_mean][i+1]) for i in unique(df[!,:p_idx])])
    p_short_cnstr = Dict([(i,pweight_df[!,:sg_short_mean][i+1]) for i in unique(df[!,:p_idx])])
    p_putt_cnstr = Dict([(i,pweight_df[!,:sg_putt_mean][i+1]) for i in unique(df[!,:p_idx])])
    p_total_cnstr = Dict([(i,pweight_df[!,:sg_total_mean][i+1]) for i in unique(df[!,:p_idx])])
    #println("test4.5")
    p_weight[!,:sg_drive_mean] = collect(values(p_drive_cnstr))
    p_weight[!,:sg_appr_mean] = collect(values(p_appr_cnstr))
    p_weight[!,:sg_short_mean] = collect(values(p_short_cnstr))
    p_weight[!,:sg_putt_mean] = collect(values(p_putt_cnstr))
    p_weight[!,:sg_total_mean] = collect(values(p_total_cnstr))
    #println("test5")
    c_drive_cnstr = Dict([(i,cweight_df[!,:sg_drive_mean][i+1]) for i in unique(df[!,:c_idx])])
    c_appr_cnstr = Dict([(i,cweight_df[!,:sg_appr_mean][i+1]) for i in unique(df[!,:c_idx])])
    c_short_cnstr = Dict([(i,cweight_df[!,:sg_short_mean][i+1]) for i in unique(df[!,:c_idx])])
    c_putt_cnstr = Dict([(i,cweight_df[!,:sg_putt_mean][i+1]) for i in unique(df[!,:c_idx])])
    c_total_cnstr = Dict([(i,cweight_df[!,:sg_total_mean][i+1]) for i in unique(df[!,:c_idx])])
    c_weight[!,:sg_drive_mean] = collect(values(c_drive_cnstr))
    c_weight[!,:sg_appr_mean] = collect(values(c_appr_cnstr))
    c_weight[!,:sg_short_mean] = collect(values(c_short_cnstr))
    c_weight[!,:sg_putt_mean] = collect(values(c_putt_cnstr))
    c_weight[!,:sg_total_mean] = collect(values(c_total_cnstr))
    
    
    return p_weight, c_weight
end

function get_weights_nodsr(df, pweight_df = pweights,cweight_df = cweights)
    #println("test1")
    n = nrow(df)
    p_d=Dict([(i,count(x->x==i,df[!,:p_idx])/n) for i in unique(df[!,:p_idx])])
    p_weight = DataFrame(p_idx = collect(keys(p_d)), weight = collect(values(p_d)))
    #println("test2")
    c_d = Dict([(i,count(x->x==i,df[!,:c_idx])/n) for i in unique(df[!,:c_idx])])
    c_weight = DataFrame(c_idx = collect(keys(c_d)), weight = collect(values(c_d)))
    #println("test3")
    
    return p_weight, c_weight
end


function test_rmse( p_sg , c_sg , c_dsr , data )
    #println("test1rmse")
    pred_data = join(data, p_sg, on = :p_idx, makeunique=true)
    #println("test2")
    pred_data = join(pred_data, c_sg, on = :c_idx, makeunique=true)
    #println("test3rmse")
    pred_data = join(pred_data, c_dsr, on = :c_idx, makeunique=true)
    #println("test4rmse")
    
    pred_data[!,:SG_drive_pred] = (pred_data[!,:sg_drive].*pred_data[!,:dsr_drive] .- pred_data[!,:sg_drive_1]) .- pred_data[!,:SG_drive]
    pred_data[!,:SG_appr_pred] = (pred_data[!,:sg_appr].*pred_data[!,:dsr_appr] .- pred_data[!,:sg_appr_1]).- pred_data[!,:SG_appr]
    pred_data[!,:SG_short_pred] = (pred_data[!,:sg_short].*pred_data[!,:dsr_short] .- pred_data[!,:sg_short_1]).- pred_data[!,:SG_short]
    pred_data[!,:SG_putt_pred] = (pred_data[!,:sg_putt].*pred_data[!,:dsr_putt] .- pred_data[!,:sg_putt_1]).- pred_data[!,:SG_putt]
    
    pred_data[!,:pred_sq_error] = pred_data[!,:SG_drive_pred].*pred_data[!,:SG_drive_pred] .+ pred_data[!,:SG_appr_pred].*pred_data[!,:SG_appr_pred] .+ pred_data[!,:SG_short_pred].*pred_data[!,:SG_short_pred] .+ pred_data[!,:SG_putt_pred].*pred_data[!,:SG_putt_pred]
    rmse = sqrt(mean(pred_data[!,:pred_sq_error]))
    drive_rmse = sqrt(mean(pred_data[!,:SG_drive_pred].*pred_data[!,:SG_drive_pred]))
    appr_rmse = sqrt(mean(pred_data[!,:SG_appr_pred].*pred_data[!,:SG_appr_pred]))
    short_rmse = sqrt(mean(pred_data[!,:SG_short_pred].*pred_data[!,:SG_short_pred]))
    putt_rmse = sqrt(mean(pred_data[!,:SG_putt_pred].*pred_data[!,:SG_putt_pred]))
    
    return rmse, drive_rmse, appr_rmse, short_rmse, putt_rmse,  pred_data[!,:pred_sq_error]
end


function test_rmse_nodsr( p_sg , c_sg  , data )
    #println("test1")
    pred_data = join(data, p_sg, on = :p_idx, makeunique=true)
    #println("test2")
    pred_data = join(pred_data, c_sg, on = :c_idx, makeunique=true)
    #println("test3")
    
    pred_data[!,:SG_drive_pred] = (pred_data[!,:sg_drive] .- pred_data[!,:sg_drive_1]) .- pred_data[!,:SG_drive]
    pred_data[!,:SG_appr_pred] = (pred_data[!,:sg_appr] .- pred_data[!,:sg_appr_1]).- pred_data[!,:SG_appr]
    pred_data[!,:SG_short_pred] = (pred_data[!,:sg_short] .- pred_data[!,:sg_short_1]).- pred_data[!,:SG_short]
    pred_data[!,:SG_putt_pred] = (pred_data[!,:sg_putt] .- pred_data[!,:sg_putt_1]).- pred_data[!,:SG_putt]
    
    pred_data[!,:pred_sq_error] = pred_data[!,:SG_drive_pred].*pred_data[!,:SG_drive_pred] .+ pred_data[!,:SG_appr_pred].*pred_data[!,:SG_appr_pred] .+ pred_data[!,:SG_short_pred].*pred_data[!,:SG_short_pred] .+ pred_data[!,:SG_putt_pred].*pred_data[!,:SG_putt_pred]
    rmse = sqrt(mean(pred_data[!,:pred_sq_error]))
    drive_rmse = sqrt(mean(pred_data[!,:SG_drive_pred].*pred_data[!,:SG_drive_pred]))
    appr_rmse = sqrt(mean(pred_data[!,:SG_appr_pred].*pred_data[!,:SG_appr_pred]))
    short_rmse = sqrt(mean(pred_data[!,:SG_short_pred].*pred_data[!,:SG_short_pred]))
    putt_rmse = sqrt(mean(pred_data[!,:SG_putt_pred].*pred_data[!,:SG_putt_pred]))
    
    return rmse, drive_rmse, appr_rmse, short_rmse, putt_rmse,  pred_data[!,:pred_sq_error]
end

function get_avg_ests(df, p_avgs = plyr_avgs,c_avgs = course_avgs)
    #println("test1")
    n = nrow(df)
    p_ests = DataFrame(p_idx = unique(df[!,:p_idx]))
    #println("test2")
    c_ests = DataFrame(c_idx = unique(df[!,:c_idx]))
    #println("test3")
    p_drive_cnstr = Dict([(i,p_avgs[!,:sg_drive_mean][i+1]) for i in unique(df[!,:p_idx])])
    #println("test4")
    p_appr_cnstr = Dict([(i,p_avgs[!,:sg_appr_mean][i+1]) for i in unique(df[!,:p_idx])])
    p_short_cnstr = Dict([(i,p_avgs[!,:sg_short_mean][i+1]) for i in unique(df[!,:p_idx])])
    p_putt_cnstr = Dict([(i,p_avgs[!,:sg_putt_mean][i+1]) for i in unique(df[!,:p_idx])])
    #println("test4.5")
    p_ests[!,:sg_drive_mean] = collect(values(p_drive_cnstr))
    p_ests[!,:sg_appr_mean] = collect(values(p_appr_cnstr))
    p_ests[!,:sg_short_mean] = collect(values(p_short_cnstr))
    p_ests[!,:sg_putt_mean] = collect(values(p_putt_cnstr))
    #println("test5")
    c_drive_cnstr = Dict([(i,c_avgs[!,:sg_drive_mean][i+1]) for i in unique(df[!,:c_idx])])
    c_appr_cnstr = Dict([(i,c_avgs[!,:sg_appr_mean][i+1]) for i in unique(df[!,:c_idx])])
    c_short_cnstr = Dict([(i,c_avgs[!,:sg_short_mean][i+1]) for i in unique(df[!,:c_idx])])
    c_putt_cnstr = Dict([(i,c_avgs[!,:sg_putt_mean][i+1]) for i in unique(df[!,:c_idx])])
    
    c_ests[!,:sg_drive_mean] = collect(values(c_drive_cnstr))
    c_ests[!,:sg_appr_mean] = collect(values(c_appr_cnstr))
    c_ests[!,:sg_short_mean] = collect(values(c_short_cnstr))
    c_ests[!,:sg_putt_mean] = collect(values(c_putt_cnstr))
    
    
    
    return p_ests, c_ests
end

function test_rmse_avg( p_sg , c_sg  , data )
    #println("test1")
    pred_data = join(data, p_sg, on = :p_idx, makeunique=true)
    #println("test2")
    pred_data = join(pred_data, c_sg, on = :c_idx, makeunique=true)
    #println("test3")
    
    pred_data[!,:SG_drive_pred] = (pred_data[!,:sg_drive_mean] .+ pred_data[!,:sg_drive_mean_1]) .- pred_data[!,:SG_drive]
    pred_data[!,:SG_appr_pred] = (pred_data[!,:sg_appr_mean] .+ pred_data[!,:sg_appr_mean_1]) .- pred_data[!,:SG_appr]
    pred_data[!,:SG_short_pred] = (pred_data[!,:sg_short_mean] .+ pred_data[!,:sg_short_mean_1]) .- pred_data[!,:SG_short]
    pred_data[!,:SG_putt_pred] = (pred_data[!,:sg_putt_mean] .+ pred_data[!,:sg_putt_mean_1]) .- pred_data[!,:SG_putt]
    
    pred_data[!,:pred_sq_error] = pred_data[!,:SG_drive_pred].*pred_data[!,:SG_drive_pred] .+ pred_data[!,:SG_appr_pred].*pred_data[!,:SG_appr_pred] .+ pred_data[!,:SG_short_pred].*pred_data[!,:SG_short_pred] .+ pred_data[!,:SG_putt_pred].*pred_data[!,:SG_putt_pred]
    rmse = sqrt(mean(pred_data[!,:pred_sq_error]))
    drive_rmse = sqrt(mean(pred_data[!,:SG_drive_pred].*pred_data[!,:SG_drive_pred]))
    appr_rmse = sqrt(mean(pred_data[!,:SG_appr_pred].*pred_data[!,:SG_appr_pred]))
    short_rmse = sqrt(mean(pred_data[!,:SG_short_pred].*pred_data[!,:SG_short_pred]))
    putt_rmse = sqrt(mean(pred_data[!,:SG_putt_pred].*pred_data[!,:SG_putt_pred]))
    
    return rmse, drive_rmse, appr_rmse, short_rmse, putt_rmse,  pred_data[!,:pred_sq_error]
end


function shuffle_data(df)
    rmd = 0
    train = DataFrame()
    test = DataFrame()
    for i = unique(df[!,:p_idx])
        unq_courses = unique(filter(row -> (row.p_idx == i) , df)[!,:c_idx])
        n = length(unq_courses)
        if n <2
            #print(i, ": excluded   ")
            rmd = rmd + 1
            continue
        else
            idx = shuffle(1:n)
            train_idx = view(idx, 1:floor(Int, train_prcnt*n))
            test_idx = view(idx, (floor(Int, train_prcnt*n)+1):n)
            train_courses = unq_courses[train_idx,:]
            
            test_courses = unq_courses[test_idx,:]
            for j = train_courses
                append!(train, filter(row -> (row.p_idx == i) & (row.c_idx ==j) , df))
            end
            for k = test_courses
                append!(test, filter(row -> (row.p_idx == i) & (row.c_idx ==k) , df))
            end
        end
    end
        
    #print("total removed: ", rmd)
    return train, test
end

shuffle_data (generic function with 1 method)

In [40]:
#train, test = shuffle_data(real_data_)

# AVG Estimates

In [73]:
real_data_ = CSV.read("C:/Users/Luke/OneDrive/Research/model_data/SYNTHETIC_realdata_07122020.csv") # scaled data

plyr_avgs = CSV.read("C:/Users/Luke/OneDrive/Research/sg_data/plyr_raw_avg.csv")
course_avgs = CSV.read("C:/Users/Luke/OneDrive/Research/sg_data/course_raw_avg.csv")

avg_pred_errors = []
avg_rmse = 0
avg_drive_rmse = 0
avg_appr_rmse= 0
avg_short_rmse = 0
avg_putt_rmse = 0

for i in 1:num_iterations
    println("Iteration: ", i)

    train_data, test_data = shuffle_data(real_data_)
    #n = nrow(real_data_)
    #idx = shuffle(1:n)
    #train_idx = view(idx, 1:floor(Int, train_prcnt*n))
    #test_idx = view(idx, (floor(Int, train_prcnt*n)+1):n)
    #train_data = real_data_[train_idx,:]
    #test_data = real_data_[test_idx,:]
    
    avg_p_final_df, avg_c_final_df = get_avg_ests(train_data)
    
    new_rmse, drive_rmse, appr_rmse, short_rmse, putt_rmse, preds = test_rmse_avg(avg_p_final_df, avg_c_final_df,  test_data)
    avg_rmse = avg_rmse + new_rmse
    avg_drive_rmse = avg_drive_rmse + drive_rmse
    avg_appr_rmse = avg_appr_rmse + appr_rmse
    avg_short_rmse = avg_short_rmse + short_rmse
    avg_putt_rmse = avg_putt_rmse + putt_rmse
    
    
    append!(avg_pred_errors, preds)
    
    println( "\nIteration RMSE: ", new_rmse)
    println("\nTotal RMSE: ", avg_rmse/i, "\n")
    println("\nDrive RMSE: ", avg_drive_rmse/i, "\n")
    println("\nAppr RMSE: ", avg_appr_rmse/i, "\n")
    println("\nShort RMSE: ", avg_short_rmse/i, "\n")
    println("\nPutt RMSE: ", avg_putt_rmse/i, "\n")
    
end

Iteration: 1

Iteration RMSE: 3.419257002793939

Total RMSE: 3.419257002793939


Drive RMSE: 1.481923189517865


Appr RMSE: 2.1330959120239896


Short RMSE: 1.2064524781214494


Putt RMSE: 1.8680461342444803

Iteration: 2

Iteration RMSE: 3.370607861586121

Total RMSE: 3.39493243219003


Drive RMSE: 1.4801664437898077


Appr RMSE: 2.105815655227584


Short RMSE: 1.1929052759317134


Putt RMSE: 1.8646286834591606

Iteration: 3

Iteration RMSE: 3.347377070397676

Total RMSE: 3.379080644925912


Drive RMSE: 1.4843344692662612


Appr RMSE: 2.084138523580959


Short RMSE: 1.1960889414635547


Putt RMSE: 1.854656306857726

Iteration: 4

Iteration RMSE: 3.345713490528709

Total RMSE: 3.370738856326611


Drive RMSE: 1.4797257519797211


Appr RMSE: 2.075222012712751


Short RMSE: 1.1954977822337658


Putt RMSE: 1.853574648884348

Iteration: 5

Iteration RMSE: 3.4056478982976035

Total RMSE: 3.3777206647208096


Drive RMSE: 1.4988115461246363


Appr RMSE: 2.0732501172535107


Short RMSE: 1.19588

# NO DSR Estimates

In [14]:
# w out dsr

real_data_ = CSV.read("C:/Users/Luke/OneDrive/Research/model_data/SYNTHETIC_realdata_07122020.csv") # scaled data
real_data_[!, :SG_total] = real_data_[!, :SG_appr] + real_data_[!, :SG_short]  +real_data_[!, :SG_drive] + real_data_[!, :SG_putt] 

nodsr_pred_errors = []
nodsr_rmse = 0
nodsr_drive_rmse = 0
nodsr_appr_rmse= 0
nodsr_short_rmse = 0
nodsr_putt_rmse = 0

nodsr_p_ests = DataFrame()
nodsr_c_ests = DataFrame()

nodsr_train_error = 0

for i in 1:num_iterations
    println("Iteration: ", i)
    
    nodsr_obj_val=0
    n = nrow(real_data_)
    #idx = shuffle(1:n)
    #train_idx = view(idx, 1:floor(Int, train_prcnt*n))
    #test_idx = view(idx, (floor(Int, train_prcnt*n)+1):n)
    #train_data = real_data_[train_idx,:]
    #test_data = real_data_[test_idx,:]
    train_data, test_data = shuffle_data(real_data_)
    
    pweights__, cweights__ = get_weights_nodsr(train_data)
    train_plyrs = unique(train_data[!,:player])
    train_courses = unique(train_data[!,:course_name])
    # need to remove any values in training but not test
    
    n_real = length(train_data[!,:player])
    num_plyrs = length(unique(train_data[!,:player]))
    num_courses = length(unique(train_data[!,:course_name]))
    
    idx_X_p_idx = DataFrame(X_p_idx = 1:num_plyrs, p_idx = unique(train_data[!,:p_idx]))
    idx_X_c_idx = DataFrame(X_c_idx = 1:num_courses, c_idx = unique(train_data[!,:c_idx]))
    
    train_data = join(train_data, idx_X_p_idx, on = :p_idx)
    real_data = join(train_data, idx_X_c_idx, on = :c_idx)
    
    pweight_df = join(pweights__, idx_X_p_idx, on = :p_idx)
    cweight_df = join(cweights__, idx_X_c_idx, on = :c_idx)

    mod_real = Model(with_optimizer(Ipopt.Optimizer))
   
    @variable(mod_real, p_sg_drive[1:num_plyrs])
    @variable(mod_real, p_sg_appr[1:num_plyrs])
    @variable(mod_real, p_sg_short[1:num_plyrs])
    @variable(mod_real, p_sg_putt[1:num_plyrs])

    @variable(mod_real, c_sg_drive[1:num_courses])
    @variable(mod_real, c_sg_appr[1:num_courses])
    @variable(mod_real, c_sg_short[1:num_courses])
    @variable(mod_real, c_sg_putt[1:num_courses])
    
    p_sg_drive_ = [p_sg_drive[i] for i in real_data[!,:X_p_idx]]
    p_sg_appr_ = [p_sg_appr[i] for i in real_data[!,:X_p_idx]]
    p_sg_short_ = [p_sg_short[i] for i in real_data[!,:X_p_idx]]
    p_sg_putt_ = [p_sg_putt[i] for i in real_data[!,:X_p_idx]]

    c_sg_drive_ = [c_sg_drive[i] for i in real_data[!,:X_c_idx]]
    c_sg_appr_ = [c_sg_appr[i] for i in real_data[!,:X_c_idx]]
    c_sg_short_ = [c_sg_short[i] for i in real_data[!,:X_c_idx]]
    c_sg_putt_ = [c_sg_putt[i] for i in real_data[!,:X_c_idx]]
    
    hsg_wmean_drive = @expression(mod_real, sum(c_sg_drive[i]*(filter(row -> (row.X_c_idx == i) , cweight_df)[!,:weight][1]) for i in 1:num_courses))
    hsg_wmean_appr = @expression(mod_real, sum(c_sg_appr[i]*(filter(row -> (row.X_c_idx == i) , cweight_df)[!,:weight][1]) for i in 1:num_courses))
    hsg_wmean_short = @expression(mod_real, sum(c_sg_short[i]*(filter(row -> (row.X_c_idx == i) , cweight_df)[!,:weight][1]) for i in 1:num_courses))
    hsg_wmean_putt = @expression(mod_real, sum(c_sg_putt[i]*(filter(row -> (row.X_c_idx == i) , cweight_df)[!,:weight][1]) for i in 1:num_courses))

    hsg_drive_avg_constr = @constraint(mod_real,  hsg_wmean_drive == 0.0) 
    hsg_appr_avg_constr = @constraint(mod_real,  hsg_wmean_appr == 0.0) 
    hsg_short_avg_constr = @constraint(mod_real,  hsg_wmean_short == 0.0) 
    hsg_putt_avg_constr = @constraint(mod_real,  hsg_wmean_putt == 0.0)

    psg_wmean_drive = @expression(mod_real, sum(p_sg_drive[i]*(filter(row -> (row.X_p_idx == i) , pweight_df)[!,:weight][1]) for i in 1:num_plyrs))
    psg_wmean_appr = @expression(mod_real, sum(p_sg_appr[i]*(filter(row -> (row.X_p_idx == i) , pweight_df)[!,:weight][1]) for i in 1:num_plyrs))
    psg_wmean_short = @expression(mod_real, sum(p_sg_short[i]*(filter(row -> (row.X_p_idx == i) , pweight_df)[!,:weight][1]) for i in 1:num_plyrs))
    psg_wmean_putt = @expression(mod_real, sum(p_sg_putt[i]*(filter(row -> (row.X_p_idx == i) , pweight_df)[!,:weight][1]) for i in 1:num_plyrs))

    psg_drive_avg_constr = @constraint(mod_real,  psg_wmean_drive == 0.0) 
    psg_appr_avg_constr = @constraint(mod_real,  psg_wmean_appr == 0.0) 
    psg_short_avg_constr = @constraint(mod_real,  psg_wmean_short == 0.0) 
    psg_putt_avg_constr = @constraint(mod_real,  psg_wmean_putt == 0.0)

    sg_drive_constr = @constraint(mod_real, [i=1:num_courses], -5 <= c_sg_drive[i] <= 5) 
    sg_appr_constr = @constraint(mod_real, [i=1:num_courses],  -5 <= c_sg_appr[i] <= 5) 
    sg_short_constr = @constraint(mod_real, [i=1:num_courses],  -5 <= c_sg_short[i] <= 5) 
    sg_putt_constr = @constraint(mod_real, [i=1:num_courses],  -5 <= c_sg_putt[i] <= 5) 

    psg_drive_constr = @constraint(mod_real, [i=1:num_plyrs], -5 <= p_sg_drive[i] <= 5) 
    psg_appr_constr = @constraint(mod_real, [i=1:num_plyrs],  -5 <= p_sg_appr[i] <= 5) 
    psg_short_constr = @constraint(mod_real, [i=1:num_plyrs],  -5 <= p_sg_short[i] <= 5) 
    psg_putt_constr = @constraint(mod_real, [i=1:num_plyrs],  -5 <= p_sg_putt[i] <= 5) 
    
    @NLobjective(mod_real, Min, sum( (p_sg_drive_[i]*p_sg_drive_[i] - 2*p_sg_drive_[i]*c_sg_drive_[i] + c_sg_drive_[i]*c_sg_drive_[i] - 2*p_sg_drive_[i]*real_data[!,:SG_drive][i] + 2*c_sg_drive_[i]*real_data[!,:SG_drive][i] + real_data[!,:SG_drive][i]*real_data[!,:SG_drive][i]) + (p_sg_appr_[i]*p_sg_appr_[i] - 2*p_sg_appr_[i]*c_sg_appr_[i] + c_sg_appr_[i]*c_sg_appr_[i] - 2*p_sg_appr_[i]*real_data[!,:SG_appr][i] + 2*c_sg_appr_[i]*real_data[!,:SG_appr][i] + real_data[!,:SG_appr][i]*real_data[!,:SG_appr][i]) + (p_sg_short_[i]*p_sg_short_[i] - 2*p_sg_short_[i]*c_sg_short_[i] + c_sg_short_[i]*c_sg_short_[i] - 2*p_sg_short_[i]*real_data[!,:SG_short][i] + 2*c_sg_short_[i]*real_data[!,:SG_short][i] + real_data[!,:SG_short][i]*real_data[!,:SG_short][i]) + (p_sg_putt_[i]*p_sg_putt_[i] - 2*p_sg_putt_[i]*c_sg_putt_[i] + c_sg_putt_[i]*c_sg_putt_[i] - 2*p_sg_putt_[i]*real_data[!,:SG_putt][i] + 2*c_sg_putt_[i]*real_data[!,:SG_putt][i] + real_data[!,:SG_putt][i]*real_data[!,:SG_putt][i]) for i in 1:n_real)/( n_real))
    status_real = optimize!(mod_real)
    
    c_idx_df = DataFrame(idx = unique(real_data[!,:c_idx]), X_idx = unique(real_data[!,:X_c_idx]), course_name = unique(real_data[!,:course_name]))
    p_idx_df = DataFrame(idx = unique(real_data[!,:p_idx]), X_idx = unique(real_data[!,:X_p_idx]), player = unique(real_data[!,:player]))

    p_d_label = [value.(p_sg_drive)[i] for i in p_idx_df[!,:X_idx]]
    p_a_label = [value.(p_sg_appr)[i] for i in p_idx_df[!,:X_idx]]
    p_s_label = [value.(p_sg_short)[i] for i in p_idx_df[!,:X_idx]]
    p_p_label = [value.(p_sg_putt)[i] for i in p_idx_df[!,:X_idx]]

    c_d_label = [value.(c_sg_drive)[i] for i in c_idx_df[!,:X_idx]]
    c_a_label = [value.(c_sg_appr)[i] for i in c_idx_df[!,:X_idx]]
    c_s_label = [value.(c_sg_short)[i] for i in c_idx_df[!,:X_idx]]
    c_p_label = [value.(c_sg_putt)[i] for i in c_idx_df[!,:X_idx]]


    p_rounds = [filter(row -> (row.X_p_idx == i) , pweight_df)[!,:weight][1]*n_real for i in p_idx_df[!,:X_idx]]
    c_rounds = [filter(row -> (row.X_c_idx == i) , cweight_df)[!,:weight][1]*n_real for i in c_idx_df[!,:X_idx]]
    nodsr_obj_val = sqrt(objective_value(mod_real))
    
    nodsr_c_sg_final_df = DataFrame(c_idx = c_idx_df[!,:idx], course_name = c_idx_df[!,:course_name], sg_drive = c_d_label, sg_appr = c_a_label, sg_short = c_s_label, sg_putt = c_p_label,num_rounds = c_rounds)

    nodsr_p_final_df = DataFrame(p_idx = p_idx_df[!,:idx], player = p_idx_df[!,:player], sg_drive = p_d_label, sg_appr = p_a_label, sg_short = p_s_label, sg_putt = p_p_label, num_rounds = p_rounds)

    
    append!(nodsr_c_ests, nodsr_c_sg_final_df)
    append!(nodsr_p_ests, nodsr_p_final_df)
    
    new_rmse, drive_rmse, appr_rmse, short_rmse, putt_rmse, preds = test_rmse_nodsr(nodsr_p_final_df, nodsr_c_sg_final_df,  test_data)
    nodsr_rmse = nodsr_rmse + new_rmse
    nodsr_drive_rmse = nodsr_drive_rmse + drive_rmse
    nodsr_appr_rmse = nodsr_appr_rmse + appr_rmse
    nodsr_short_rmse = nodsr_short_rmse + short_rmse
    nodsr_putt_rmse = nodsr_putt_rmse + putt_rmse
    
    
    println( "\nIteration RMSE: ", new_rmse)
    
    append!(nodsr_pred_errors, preds)
    nodsr_train_error =  nodsr_train_error  + nodsr_obj_val
    
    println("Iteration ",i,"\nTraining error: ", sqrt(nodsr_obj_val))
    println()
    println("Iteration RMSE: ", new_rmse)
    println("Average RMSES: \n")
    println("\nTotal RMSE: ", nodsr_rmse/i, "\n")
    println("\nDrive RMSE: ", nodsr_drive_rmse/i, "\n")
    println("\nAppr RMSE: ", nodsr_appr_rmse/i, "\n")
    println("\nShort RMSE: ", nodsr_short_rmse/i, "\n")
    println("\nPutt RMSE: ", nodsr_putt_rmse/i, "\n")
    
end

Iteration: 1
This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      236
Number of nonzeros in inequality constraint Jacobian.:      472
Number of nonzeros in Lagrangian Hessian.............:      688

Total number of variables............................:      236
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        8
Total number of inequality constraints...............:      472
        inequality constraints with only lower bounds:      236
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      236

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha

Iteration RMSE: 0.5074464509277841
Iteration 3
Training error: 0.7098727216082895

Iteration RMSE: 0.5074464509277841
Average RMSES: 


Total RMSE: 0.4030600477209035


Drive RMSE: 0.19467463385984526


Appr RMSE: 0.20912849908899578


Short RMSE: 0.20512964240825862


Putt RMSE: 0.1965430207187913

Iteration: 4
This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      232
Number of nonzeros in inequality constraint Jacobian.:      464
Number of nonzeros in Lagrangian Hessian.............:      684

Total number of variables............................:      232
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        8
Total number of inequality co

Total RMSE: 0.28235059534747414


Drive RMSE: 0.13692840432446243


Appr RMSE: 0.14558665824688016


Short RMSE: 0.14597199877103198


Putt RMSE: 0.13502155726478704

Iteration: 7
This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      236
Number of nonzeros in inequality constraint Jacobian.:      472
Number of nonzeros in Lagrangian Hessian.............:      688

Total number of variables............................:      236
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        8
Total number of inequality constraints...............:      472
        inequality constraints with only lower bounds:      236
   inequality constraints with lowe

Iteration RMSE: 0.1055927179228715
Iteration 9
Training error: 0.3229237824719703

Iteration RMSE: 0.1055927179228715
Average RMSES: 


Total RMSE: 0.23265979257276564


Drive RMSE: 0.11402952408330085


Appr RMSE: 0.11916313723691863


Short RMSE: 0.12251214831957952


Putt RMSE: 0.10744566458825101

Iteration: 10
This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      236
Number of nonzeros in inequality constraint Jacobian.:      472
Number of nonzeros in Lagrangian Hessian.............:      688

Total number of variables............................:      236
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        8
Total number of inequality

Iteration RMSE: 0.2164705534340568
Average RMSES: 


Total RMSE: 0.23194650226545374


Drive RMSE: 0.11399519641561445


Appr RMSE: 0.11840475461802334


Short RMSE: 0.12321104055308234


Putt RMSE: 0.10618193902950834

Iteration: 13
This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      232
Number of nonzeros in inequality constraint Jacobian.:      464
Number of nonzeros in Lagrangian Hessian.............:      684

Total number of variables............................:      232
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        8
Total number of inequality constraints...............:      464
        inequality constraints with only lowe

Iteration: 16
This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      228
Number of nonzeros in inequality constraint Jacobian.:      456
Number of nonzeros in Lagrangian Hessian.............:      680

Total number of variables............................:      228
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        8
Total number of inequality constraints...............:      456
        inequality constraints with only lower bounds:      228
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      228

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alph

Iteration: 19
This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      232
Number of nonzeros in inequality constraint Jacobian.:      464
Number of nonzeros in Lagrangian Hessian.............:      684

Total number of variables............................:      232
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        8
Total number of inequality constraints...............:      464
        inequality constraints with only lower bounds:      232
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      232

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alph

In [6]:
CSV.write("model_data/TESTSET_nodsr_p_ests.csv", nodsr_p_ests)
CSV.write("model_data/TESTSET_nodsr_c_ests.csv", nodsr_c_ests)

"model_data/TESTSET_nodsr_c_ests.csv"

# DSR ESTIMATES

In [15]:
real_data_ = CSV.read("C:/Users/Luke/OneDrive/Research/model_data/SYNTHETIC_realdata_07122020.csv") # scaled data
real_data_[!, :SG_total] = real_data_[!, :SG_appr] + real_data_[!, :SG_short]  +real_data_[!, :SG_drive] + real_data_[!, :SG_putt] 

#p_df = CSV.read("test_pdf_0623.csv")
#c_df = CSV.read("test_cdf_0623.csv")
#c_dsr_df = CSV.read("test_cdsrdf_0623.csv")


pred_errors = []
dsr_rmse = 0
dsr_drive_rmse = 0
dsr_appr_rmse= 0
dsr_short_rmse = 0
dsr_putt_rmse = 0

dsr_p_ests = DataFrame()
dsr_c_ests = DataFrame()
dsr_d_ests = DataFrame()

dsr_train_error = 0


# w dsr

for i in 1:num_iterations
    
    println("Iteration: ", i)
    
    obj_val=0
    
    train_data, test_data = shuffle_data(real_data_)
    
    pweights__, cweights__ = get_weights(train_data)
    
    n_real = length(train_data[!,:player])
    num_plyrs = length(unique(train_data[!,:player]))
    num_courses = length(unique(train_data[!,:course_name]))
    
    idx_X_p_idx = DataFrame(X_p_idx = 1:num_plyrs, p_idx = unique(train_data[!,:p_idx]))
    idx_X_c_idx = DataFrame(X_c_idx = 1:num_courses, c_idx = unique(train_data[!,:c_idx]))
   # println("here3")
    train2_data = join(train_data, idx_X_p_idx, on = :p_idx)
    real_data = join(train2_data, idx_X_c_idx, on = :c_idx)
    #println("here2")
    pweight_df = join(pweights__, idx_X_p_idx, on = :p_idx)
    cweight_df = join(cweights__, idx_X_c_idx, on = :c_idx)
    
    # driver
    #println("here1")
    mod_real = Model(with_optimizer(Ipopt.Optimizer))

    @variable(mod_real, p_sg_drive[1:num_plyrs])
    @variable(mod_real, c_sg_drive[1:num_courses])
    @variable(mod_real, c_dsr_drive[1:num_courses])
    @variable(mod_real, p_sg_appr[1:num_plyrs])
    @variable(mod_real, c_sg_appr[1:num_courses])
    @variable(mod_real, c_dsr_appr[1:num_courses])
    @variable(mod_real, p_sg_short[1:num_plyrs])
    @variable(mod_real, c_sg_short[1:num_courses])
    @variable(mod_real, c_dsr_short[1:num_courses])
    @variable(mod_real, p_sg_putt[1:num_plyrs])
    @variable(mod_real, c_sg_putt[1:num_courses])
    @variable(mod_real, c_dsr_putt[1:num_courses])

    p_sg_drive_ = [p_sg_drive[i] for i in real_data[!,:X_p_idx]]
    c_sg_drive_ = [c_sg_drive[i] for i in real_data[!,:X_c_idx]]
    c_dsr_drive_ = [c_dsr_drive[i] for i in real_data[!,:X_c_idx]]
    p_sg_appr_ = [p_sg_appr[i] for i in real_data[!,:X_p_idx]]
    c_sg_appr_ = [c_sg_appr[i] for i in real_data[!,:X_c_idx]]
    c_dsr_appr_ = [c_dsr_appr[i] for i in real_data[!,:X_c_idx]]
    p_sg_short_ = [p_sg_short[i] for i in real_data[!,:X_p_idx]]
    c_sg_short_ = [c_sg_short[i] for i in real_data[!,:X_c_idx]]
    c_dsr_short_ = [c_dsr_short[i] for i in real_data[!,:X_c_idx]]
    p_sg_putt_ = [p_sg_putt[i] for i in real_data[!,:X_p_idx]]
    c_sg_putt_ = [c_sg_putt[i] for i in real_data[!,:X_c_idx]]
    c_dsr_putt_ = [c_dsr_putt[i] for i in real_data[!,:X_c_idx]]
    
   # println("here2")
    dsr_wmean_drive = @expression(mod_real, sum(c_dsr_drive[i]*(filter(row -> (row.X_c_idx == i) , cweight_df)[!,:weight][1]) for i in 1:num_courses))
    dsr_drive_avg_constr = @constraint(mod_real,  dsr_wmean_drive == 1.0) 

    hsg_wmean_drive = @expression(mod_real, sum(c_sg_drive[i]*(filter(row -> (row.X_c_idx == i) , cweight_df)[!,:weight][1]) for i in 1:num_courses))
    hsg_drive_avg_constr = @constraint(mod_real,  hsg_wmean_drive == 0.0) 

    psg_wmean_drive = @expression(mod_real, sum(p_sg_drive[i]*(filter(row -> (row.X_p_idx == i) , pweight_df)[!,:weight][1]) for i in 1:num_plyrs))
    psg_drive_avg_constr = @constraint(mod_real,  psg_wmean_drive == 0.0) 

    dsr_drive_constr = @constraint(mod_real, [i=1:num_courses], 0.5 <= c_dsr_drive[i] <= 1.5) 

    sg_drive_apx_constr = @constraint(mod_real, [i=1:num_courses], filter(row -> (row.X_c_idx == i) , cweight_df)[!,:sg_drive_mean][1] -2 <= c_sg_drive[i] <= filter(row -> (row.X_c_idx == i) , cweight_df)[!,:sg_drive_mean][1] + 2) 
    psg_drive_apx_constr = @constraint(mod_real, [i=1:num_plyrs], filter(row -> (row.X_p_idx == i) , pweight_df)[!,:sg_drive_mean][1]-2 <= p_sg_drive[i] <= filter(row -> (row.X_p_idx == i) , pweight_df)[!,:sg_drive_mean][1]+2)

    dsr_wmean_appr = @expression(mod_real, sum(c_dsr_appr[i]*(filter(row -> (row.X_c_idx == i) , cweight_df)[!,:weight][1]) for i in 1:num_courses))
    dsr_appr_avg_constr = @constraint(mod_real,  dsr_wmean_appr == 1.0) 

    hsg_wmean_appr = @expression(mod_real, sum(c_sg_appr[i]*(filter(row -> (row.X_c_idx == i) , cweight_df)[!,:weight][1]) for i in 1:num_courses))
    hsg_appr_avg_constr = @constraint(mod_real,  hsg_wmean_appr == 0.0) 

    psg_wmean_appr = @expression(mod_real, sum(p_sg_appr[i]*(filter(row -> (row.X_p_idx == i) , pweight_df)[!,:weight][1]) for i in 1:num_plyrs))
    psg_appr_avg_constr = @constraint(mod_real,  psg_wmean_appr == 0.0) 

    dsr_appr_constr = @constraint(mod_real, [i=1:num_courses], 0.5 <= c_dsr_appr[i] <= 1.5) 
    
    sg_appr_apx_constr = @constraint(mod_real, [i=1:num_courses], filter(row -> (row.X_c_idx == i) , cweight_df)[!,:sg_appr_mean][1] -2 <= c_sg_appr[i] <= filter(row -> (row.X_c_idx == i) , cweight_df)[!,:sg_appr_mean][1] + 2) 
    psg_appr_apx_constr = @constraint(mod_real, [i=1:num_plyrs], filter(row -> (row.X_p_idx == i) , pweight_df)[!,:sg_appr_mean][1]-2 <= p_sg_appr[i] <= filter(row -> (row.X_p_idx == i) , pweight_df)[!,:sg_appr_mean][1] +2)

    
    dsr_wmean_short = @expression(mod_real, sum(c_dsr_short[i]*(filter(row -> (row.X_c_idx == i) , cweight_df)[!,:weight][1]) for i in 1:num_courses))
    dsr_short_avg_constr = @constraint(mod_real,  dsr_wmean_short == 1.0) 

    hsg_wmean_short = @expression(mod_real, sum(c_sg_short[i]*(filter(row -> (row.X_c_idx == i) , cweight_df)[!,:weight][1]) for i in 1:num_courses))
    hsg_short_avg_constr = @constraint(mod_real,  hsg_wmean_short == 0.0) 

    psg_wmean_short = @expression(mod_real, sum(p_sg_short[i]*(filter(row -> (row.X_p_idx == i) , pweight_df)[!,:weight][1]) for i in 1:num_plyrs))
    psg_short_avg_constr = @constraint(mod_real,  psg_wmean_short == 0.0) 

    dsr_short_constr = @constraint(mod_real, [i=1:num_courses], 0.5 <= c_dsr_short[i] <= 1.5) 

    sg_short_apx_constr = @constraint(mod_real, [i=1:num_courses], filter(row -> (row.X_c_idx == i) , cweight_df)[!,:sg_short_mean][1] -2 <= c_sg_short[i] <= filter(row -> (row.X_c_idx == i) , cweight_df)[!,:sg_short_mean][1] +2) 
    psg_short_apx_constr = @constraint(mod_real, [i=1:num_plyrs], filter(row -> (row.X_p_idx == i) , pweight_df)[!,:sg_short_mean][1]-2 <= p_sg_short[i] <= filter(row -> (row.X_p_idx == i) , pweight_df)[!,:sg_short_mean][1]+2)

    dsr_wmean_putt = @expression(mod_real, sum(c_dsr_putt[i]*(filter(row -> (row.X_c_idx == i) , cweight_df)[!,:weight][1]) for i in 1:num_courses))
    dsr_putt_avg_constr = @constraint(mod_real,  dsr_wmean_putt == 1.0) 

    hsg_wmean_putt = @expression(mod_real, sum(c_sg_putt[i]*(filter(row -> (row.X_c_idx == i) , cweight_df)[!,:weight][1]) for i in 1:num_courses))
    hsg_putt_avg_constr = @constraint(mod_real,  hsg_wmean_putt == 0.0) 

    psg_wmean_putt = @expression(mod_real, sum(p_sg_putt[i]*(filter(row -> (row.X_p_idx == i) , pweight_df)[!,:weight][1]) for i in 1:num_plyrs))
    psg_putt_avg_constr = @constraint(mod_real,  psg_wmean_putt == 0.0) 

    dsr_putt_constr = @constraint(mod_real, [i=1:num_courses], 0.5 <= c_dsr_putt[i] <= 1.5) 
    
    sg_putt_apx_constr = @constraint(mod_real, [i=1:num_courses], filter(row -> (row.X_c_idx == i) , cweight_df)[!,:sg_putt_mean][1] -2 <= c_sg_putt[i] <= filter(row -> (row.X_c_idx == i) , cweight_df)[!,:sg_putt_mean][1] + 2) 
    psg_putt_apx_constr = @constraint(mod_real, [i=1:num_plyrs], filter(row -> (row.X_p_idx == i) , pweight_df)[!,:sg_putt_mean][1]-2 <= p_sg_putt[i] <= filter(row -> (row.X_p_idx == i) , pweight_df)[!,:sg_putt_mean][1]+2)

    
    # p*d - c
    @NLobjective(mod_real, Min, sum( (p_sg_drive_[i]*p_sg_drive_[i]*c_dsr_drive_[i]*c_dsr_drive_[i] - 2*p_sg_drive_[i]*c_dsr_drive_[i]*c_sg_drive_[i] + c_sg_drive_[i]*c_sg_drive_[i]- 2*p_sg_drive_[i]*c_dsr_drive_[i]*real_data[!,:SG_drive][i] + 2*c_sg_drive_[i]*real_data[!,:SG_drive][i] + real_data[!,:SG_drive][i]*real_data[!,:SG_drive][i]) + (p_sg_appr_[i]*p_sg_appr_[i]*c_dsr_appr_[i]*c_dsr_appr_[i] - 2*p_sg_appr_[i]*c_dsr_appr_[i]*c_sg_appr_[i] + c_sg_appr_[i]*c_sg_appr_[i]- 2*p_sg_appr_[i]*c_dsr_appr_[i]*real_data[!,:SG_appr][i] + 2*c_sg_appr_[i]*real_data[!,:SG_appr][i] + real_data[!,:SG_appr][i]*real_data[!,:SG_appr][i]) + (p_sg_short_[i]*p_sg_short_[i]*c_dsr_short_[i]*c_dsr_short_[i] - 2*p_sg_short_[i]*c_dsr_short_[i]*c_sg_short_[i] + c_sg_short_[i]*c_sg_short_[i]- 2*p_sg_short_[i]*c_dsr_short_[i]*real_data[!,:SG_short][i] + 2*c_sg_short_[i]*real_data[!,:SG_short][i] + real_data[!,:SG_short][i]*real_data[!,:SG_short][i]) + (p_sg_putt_[i]*p_sg_putt_[i]*c_dsr_putt_[i]*c_dsr_putt_[i] - 2*p_sg_putt_[i]*c_dsr_putt_[i]*c_sg_putt_[i] + c_sg_putt_[i]*c_sg_putt_[i]- 2*p_sg_putt_[i]*c_dsr_putt_[i]*real_data[!,:SG_putt][i] + 2*c_sg_putt_[i]*real_data[!,:SG_putt][i] + real_data[!,:SG_putt][i]*real_data[!,:SG_putt][i])  for i in 1:n_real)/(n_real))

    status_real = optimize!(mod_real)

    c_idx_df = DataFrame(idx = unique(real_data[!,:c_idx]),X_idx = unique(real_data[!,:X_c_idx]), course_name = unique(real_data[!,:course_name]))
    p_idx_df = DataFrame(idx = unique(real_data[!,:p_idx]), X_idx = unique(real_data[!,:X_p_idx]), player = unique(real_data[!,:player]))

    p_d_label = [value.(p_sg_drive)[i] for i in p_idx_df[!,:X_idx]]
    c_d_label = [value.(c_sg_drive)[i] for i in c_idx_df[!,:X_idx]]
    cd_d_label = [value.(c_dsr_drive)[i] for i in c_idx_df[!,:X_idx]]
    
    p_a_label = [value.(p_sg_appr)[i] for i in p_idx_df[!,:X_idx]]
    c_a_label = [value.(c_sg_appr)[i] for i in c_idx_df[!,:X_idx]]
    cd_a_label = [value.(c_dsr_appr)[i] for i in c_idx_df[!,:X_idx]]
    
    p_s_label = [value.(p_sg_short)[i] for i in p_idx_df[!,:X_idx]]
    c_s_label = [value.(c_sg_short)[i] for i in c_idx_df[!,:X_idx]]
    cd_s_label = [value.(c_dsr_short)[i] for i in c_idx_df[!,:X_idx]]
    
    p_p_label = [value.(p_sg_putt)[i] for i in p_idx_df[!,:X_idx]]
    c_p_label = [value.(c_sg_putt)[i] for i in c_idx_df[!,:X_idx]]
    cd_p_label = [value.(c_dsr_putt)[i] for i in c_idx_df[!,:X_idx]]

    rounds = [filter(row -> (row.X_p_idx == i) , pweight_df)[!,:weight][1]*n_real for i in p_idx_df[!,:X_idx]]
    obj_val = obj_val + objective_value(mod_real)

    c_sg_final_df = DataFrame(c_idx = c_idx_df[!,:idx], course_name = c_idx_df[!,:course_name], sg_drive = c_d_label, sg_appr = c_a_label, sg_short = c_s_label, sg_putt = c_p_label)
    c_dsr_final_df = DataFrame(c_idx = c_idx_df[!,:idx], course_name = c_idx_df[!,:course_name], dsr_drive = cd_d_label, dsr_appr = cd_a_label, dsr_short = cd_s_label, dsr_putt = cd_p_label)

    p_final_df = DataFrame(p_idx = p_idx_df[!,:idx], player = p_idx_df[!,:player], sg_drive = p_d_label,sg_appr = p_a_label, sg_short = p_s_label, sg_putt = p_p_label, num_rounds = rounds)


    obj_val = sqrt(obj_val)
    
    dsr_train_error = dsr_train_error + obj_val
    
    
    append!(dsr_c_ests, c_sg_final_df)
    append!(dsr_p_ests, p_final_df)
    append!(dsr_d_ests, c_dsr_final_df)
    
    new_rmse, drive_rmse, appr_rmse, short_rmse, putt_rmse, preds = test_rmse(p_final_df, c_sg_final_df, c_dsr_final_df, test_data)
    dsr_rmse = dsr_rmse + new_rmse
    dsr_drive_rmse = dsr_drive_rmse + drive_rmse
    dsr_appr_rmse = dsr_appr_rmse + appr_rmse
    dsr_short_rmse = dsr_short_rmse + short_rmse
    dsr_putt_rmse = dsr_putt_rmse + putt_rmse
    
    append!(pred_errors, preds)
    
    println("Iteration ",i)
    println()
    println("\nIteration RMSE: ", new_rmse)
    println("Training Error: ", dsr_train_error/i)
    println("Average RMSES: ")
    println("\nTotal RMSE: ", dsr_rmse/i)
    println("\nDrive RMSE: ", dsr_drive_rmse/i)
    println("\nAppr RMSE: ", dsr_appr_rmse/i )
    println("\nShort RMSE: ", dsr_short_rmse/i)
    println("\nPutt RMSE: ", dsr_putt_rmse/i)

            end 

Iteration: 1
This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      384
Number of nonzeros in inequality constraint Jacobian.:      768
Number of nonzeros in Lagrangian Hessian.............:     1440

Total number of variables............................:      384
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       12
Total number of inequality constraints...............:      768
        inequality constraints with only lower bounds:      384
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      384

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      384
Number of nonzeros in inequality constraint Jacobian.:      768
Number of nonzeros in Lagrangian Hessian.............:     1440

Total number of variables............................:      384
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       12
Total number of inequality constraints...............:      768
        inequality constraints with only lower bounds:      384
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      384

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      376
Number of nonzeros in inequality constraint Jacobian.:      752
Number of nonzeros in Lagrangian Hessian.............:     1428

Total number of variables............................:      376
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       12
Total number of inequality constraints...............:      752
        inequality constraints with only lower bounds:      376
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      376

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

Iteration: 10
This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      392
Number of nonzeros in inequality constraint Jacobian.:      784
Number of nonzeros in Lagrangian Hessian.............:     1452

Total number of variables............................:      392
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       12
Total number of inequality constraints...............:      784
        inequality constraints with only lower bounds:      392
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      392

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alph

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      384
Number of nonzeros in inequality constraint Jacobian.:      768
Number of nonzeros in Lagrangian Hessian.............:     1440

Total number of variables............................:      384
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       12
Total number of inequality constraints...............:      768
        inequality constraints with only lower bounds:      384
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      384

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

Total RMSE: 0.9058993697394314

Drive RMSE: 0.447553686066974

Appr RMSE: 0.413972289028457

Short RMSE: 0.45433664596238643

Putt RMSE: 0.4867974446033243
Iteration: 16
This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      376
Number of nonzeros in inequality constraint Jacobian.:      752
Number of nonzeros in Lagrangian Hessian.............:     1428

Total number of variables............................:      376
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       12
Total number of inequality constraints...............:      752
        inequality constraints with only lower bounds:      376
   inequality constraints with lower and uppe

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      384
Number of nonzeros in inequality constraint Jacobian.:      768
Number of nonzeros in Lagrangian Hessian.............:     1440

Total number of variables............................:      384
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       12
Total number of inequality constraints...............:      768
        inequality constraints with only lower bounds:      384
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      384

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

In [8]:
CSV.write("model_data/TESTSET_dsr_p_ests.csv", dsr_p_ests)
CSV.write("model_data/TESTSET_dsr_c_ests.csv", dsr_c_ests)
CSV.write("model_data/TESTSET_dsr_d_ests.csv", dsr_d_ests)

"model_data/TESTSET_dsr_d_ests.csv"

# Analysis 

In [16]:
function one_side_ttest( hyp_larger , hyp_smaller )
    diff = hyp_larger .- hyp_smaller
    test_val = (Statistics.mean(diff)*(sqrt(length(diff))))/(Statistics.std(diff))
    
    return test_val
end

one_side_ttest (generic function with 1 method)

In [17]:
min_n = min(length(nodsr_pred_errors),length(pred_errors))
dsr_ = pred_errors[1:min_n]
nodsr_ = nodsr_pred_errors[1:min_n]


print("Average train error for nodsr model: ", nodsr_train_error/num_iterations, "\n")
print("Average train error for dsr model: ", dsr_train_error/num_iterations, "\n\n")

print("Average prediction test error for nodsr model: ", nodsr_rmse/num_iterations, "\n")
print("Average prediction test error for dsr model: ", dsr_rmse/num_iterations, "\n\n")
#print("Average prediction test error for avg model: ", avg_rmse/num_iterations, "\n\n")

print("Average prediction test DRIVE error for nodsr model: ", nodsr_drive_rmse/num_iterations, "\n")
print("Average prediction test DRIVE error for dsr model: ", dsr_drive_rmse/num_iterations, "\n\n")
#print("Average prediction test DRIVE error for avg model: ", avg_drive_rmse/num_iterations, "\n\n")

print("Average prediction test APPR error for nodsr model: ", nodsr_appr_rmse/num_iterations, "\n")
print("Average prediction test APPR error for dsr model: ", dsr_appr_rmse/num_iterations, "\n\n")
#print("Average prediction test APPR error for avg model: ", avg_appr_rmse/num_iterations, "\n\n")

print("Average prediction test SHORT error for nodsr model: ", nodsr_short_rmse/num_iterations, "\n")
print("Average prediction test SHORT error for dsr model: ", dsr_short_rmse/num_iterations, "\n\n")
#print("Average prediction test SHORT error for avg model: ", avg_short_rmse/num_iterations, "\n\n")

print("Average prediction test PUTT error for nodsr model: ", nodsr_putt_rmse/num_iterations, "\n")
print("Average prediction test PUTT error for dsr model: ", dsr_putt_rmse/num_iterations, "\n\n")
#print("Average prediction test PUTT error for avg model: ", avg_putt_rmse/num_iterations, "\n\n")
print("One Side Hypothesis Tests\nCompare to critical value of 1.64 (one-sided, 0.05 significance)): \n\n")
print("Test: dsr_rmse > nodsr_rmse: \n")
print("Test Statistic: ", one_side_ttest(dsr_,nodsr_))
print("\nConclusion: nodsr model predicts significantly better, on average, compared to dsr model")
print("\n\n")


Average train error for nodsr model: 0.23344528855479982
Average train error for dsr model: 0.2532335591664922

Average prediction test error for nodsr model: 0.2344173284699355
Average prediction test error for dsr model: 0.9139680898695287

Average prediction test DRIVE error for nodsr model: 0.11505953126329363
Average prediction test DRIVE error for dsr model: 0.45164490746066593

Average prediction test APPR error for nodsr model: 0.11931401032312412
Average prediction test APPR error for dsr model: 0.4163078588172965

Average prediction test SHORT error for nodsr model: 0.12418171976059145
Average prediction test SHORT error for dsr model: 0.4597572650329832

Average prediction test PUTT error for nodsr model: 0.10820467024804148
Average prediction test PUTT error for dsr model: 0.4914018775458632

One Side Hypothesis Tests
Compare to critical value of 1.64 (one-sided, 0.05 significance)): 

Test: dsr_rmse > nodsr_rmse: 
Test Statistic: 16.67260832103974
Conclusion: nodsr model p